In [ ]:
import os
import sys
import pickle
import numpy as np
import keras_tuner as kt
from tensorflow import keras

from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.preprocessing import image

from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split

In [3]:
with open("../../pickles/X_train_aug.pkl", "rb") as file:
    X_train = pickle.load(file)

with open("../../pickles/y_train_aug.pkl", "rb") as file:
    y_train = pickle.load(file)

with open("../../pickles/labels.pkl", "rb") as file:
    labels = pickle.load(file)

# with open("../../pickles/resized_images.pkl", "rb") as file:
#     resized = pickle.load(file)




In [ ]:
print(labels)
print(labels.count) 
print(X_train[0])
print(y_train[0])
print(X_train.shape)
print(y_train.shape)    
print(X_train.shape[1:]) 

In [ ]:
print(np.unique(y_train))  # Ensure this matches expected labels (0 to 5)
count = np.unique(y_train, return_counts=True)
print(count) 

In [ ]:

# Split the training dataset into training and validation sets
X_train, X_test, y_train, y_test = train_test_split(X_train, y_train, test_size=0.2, random_state=42)

In [ ]:
def build_cnn_model(hp):
    model = keras.Sequential([
        keras.layers.Conv2D(
            filters=hp.Int('filters', min_value=32, max_value=128, step=32),
            kernel_size=hp.Choice('kernel_size', values=[3, 5]),
            activation='relu',
            input_shape=(150, 150, 3)  # Modify for your dataset
        ),
        keras.layers.MaxPooling2D(pool_size=(2, 2)),
        keras.layers.Flatten(),
        keras.layers.Dense(
            units=hp.Int('units', min_value=32, max_value=128, step=32),
            activation='relu'
        ),
        keras.layers.Dense(6, activation='softmax')  
    ])

    model.compile(
        optimizer=keras.optimizers.Adam(
            learning_rate=hp.Float('learning_rate', min_value=1e-4, max_value=1e-2, sampling='LOG')
        ),
        loss='categorical_crossentropy',
        metrics=['accuracy']
    )

    return model

# Initialize the RandomSearch Tuner
tuner = kt.RandomSearch(
    hypermodel=build_cnn_model,
    objective='val_accuracy',
    max_trials=10,
    executions_per_trial=1,  
    directory='hyperparameter_tuning',
    project_name='cnn_randomsearch'
)


# Start hyperparameter tuning
tuner.search(X_train, y_train, validation_data=(X_train, y_train), epochs=10, batch_size=32)

# Get the best hyperparameters
best_hps = tuner.get_best_hyperparameters(num_trials=1)[0]
print("Best hyperparameters:", best_hps.values)

# Retrieve and build the best model
model = tuner.hypermodel.build(best_hps)
model.summary()

In [ ]:
# Compile the model
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

In [ ]:
# Train the model
batch_size = 32
epochs = 10
history = model.fit(
    X_train, y_train,
    validation_data=(X_test, y_test),
    epochs=epochs
)

In [ ]:
# Save the model
model.save("../../models/keras_model.h5")

In [ ]:
# Test code
img_path = "../../model/pred/93.jpg"  # Change this to your image path
img_path = "../../resources/buildings/66.jpg"  # Change this to your image path

img = image.load_img(img_path, target_size=(150, 150))  # Resize to match model input size
img.show()
img_array = image.img_to_array(img)
img_array = img_array / 255.0  # Normalize if the model was trained with normalized images

# Expand dimensions to match the batch shape (model expects a batch, not a single image)
img_array = np.expand_dims(img_array, axis=0)
print(img_array.shape)
print(img_array.dtype)
# Make prediction
predictions = model.predict(img_array)
predicted_class = np.argmax(predictions)
# Print predictions
text = predictions[0][predicted_class]
print(predictions)
print(predicted_class)

In [ ]:
predicted_class = np.argmax(predictions, axis=1)  # Get class index
print("Predicted Class:", predicted_class)